## Comparison between xmask and pymask machines

### Imports modules

In [14]:
import numpy as np
import xtrack as xt
import xpart as xp
import pandas as pd
import json

# Plotting function
from plotting_functions import plot_all

# Plotly renderer
import plotly.io as pio
# ! Renderer should be set to 'notebook' or 'vscode' (depending on the IDE used) for interactive plots
pio.renderers.default = "browser"

### Build pymask line

In [15]:
path = "pymask/xsuite_lines/line_bb_for_tracking.json"
with open(path) as fid:
    dd = json.load(fid)
p_co = xp.Particles.from_dict(dd["particle_on_tracker_co"])
pm_line = xt.Line.from_dict(dd)
pm_line.particle_ref = xp.Particles.from_dict(dd["particle_on_tracker_co"])
pm_line.build_tracker()

Done loading line from dict.           
Compiling ContextCpu kernels...


ec5b66a367a749328ff9cf912d9f904e.c:4280: warning: "_GNU_SOURCE" redefined
 4280 | #  define _GNU_SOURCE // enable GNU libc NAN extension if possible
      | 
In file included from ec5b66a367a749328ff9cf912d9f904e.c:50:
/home/cdroin/miniconda3/include/python3.10/pyconfig.h:1621: note: this is the location of the previous definition
 1621 | #define _GNU_SOURCE 1
      | 


Done compiling ContextCpu kernels.


### Build xmask line

In [16]:
collider = xt.Multiline.from_json("xmask/xsuite_lines/collider_03_tuned_bb_on.json")
collider.build_trackers()
xm_line = collider["lhcb2"]

Done loading line from dict.           
Done loading line from dict.           
Done loading line from dict.           
Done loading line from dict.           
Compiling ContextCpu kernels...


29f52f651aa14c5da5de550d7ccad548.c:4280: warning: "_GNU_SOURCE" redefined
 4280 | #  define _GNU_SOURCE // enable GNU libc NAN extension if possible
      | 
In file included from 29f52f651aa14c5da5de550d7ccad548.c:50:
/home/cdroin/miniconda3/include/python3.10/pyconfig.h:1621: note: this is the location of the previous definition
 1621 | #define _GNU_SOURCE 1
      | 


Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...


94ffac7971a34524a47b4e97a2d57fec.c:4280: warning: "_GNU_SOURCE" redefined
 4280 | #  define _GNU_SOURCE // enable GNU libc NAN extension if possible
      | 
In file included from 94ffac7971a34524a47b4e97a2d57fec.c:50:
/home/cdroin/miniconda3/include/python3.10/pyconfig.h:1621: note: this is the location of the previous definition
 1621 | #define _GNU_SOURCE 1
      | 


Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.
Compiling ContextCpu kernels...
Done compiling ContextCpu kernels.


### Compare elements in both lines

#### Check element names which differ


In [8]:
# Get element names in pymask and xmask
set_pm = set(pm_line.element_names)
set_xm = set(xm_line.element_names)

# Get elements in pymask but not in xmask
set_pm_not_xm = set_pm - set_xm
print("Elements in pymask but not in xmask", set_pm_not_xm)
print("Elements in xmask but not in pymask that are not drifts", [x for x in set_pm_not_xm if 'drift' not in x])

# Get elements in xmask but not in pymask
set_xm_not_pm = set_xm - set_pm
print("Elements in pymask but not in xmask", set_xm_not_pm)
print("Elements in pymask but not in xmask", [x for x in set_xm_not_pm if 'drift' not in x])


Elements in pymask but not in xmask {'drift_11978', 'drift_11864', 'drift_2949', 'drift_8857', 'drift_10306', 'drift_11800', 'drift_11807', 'drift_11908', 'drift_11979', 'drift_11879', 'drift_11976', 'drift_8771', 'drift_11933', 'drift_10285', 'drift_11870', 'drift_2774', 'drift_10312', 'drift_11777', 'drift_11971', 'drift_8723', 'drift_11860', 'drift_2894', 'drift_11953', 'drift_11907', 'drift_8578', 'drift_11905', 'drift_10386', 'drift_8787', 'drift_10371', 'drift_11981', 'drift_11982', 'drift_11966', 'drift_11974', 'drift_10249', 'drift_11797', 'drift_2744', 'drift_11788', 'drift_11779', 'drift_11793', 'drift_7241', 'drift_11865', 'drift_11987', 'drift_11983', 'drift_11921', 'drift_11949', 'drift_11771', 'drift_10221', 'drift_7144', 'drift_11880', 'drift_11839', 'drift_11850', 'drift_11919', 'drift_11916', 'drift_11811', 'drift_11968', 'drift_8810', 'drift_11832', 'drift_11828', 'drift_10357', 'drift_11943', 'drift_11857', 'drift_3003', 'drift_11805', 'drift_11783', 'drift_10222', '

#### Check if some dipoles differ


In [9]:
dipole_names = [x for x in pm_line.element_names if 'mb' in x]
for i, dipole in enumerate(dipole_names):
    class_element = pm_line[dipole].to_dict()['__class__']
    if class_element != 'Marker' and class_element != 'Drift':
        try:
            if class_element == 'SRotation':
                assert pm_line[dipole].cos_z == xm_line[dipole].cos_z
                assert pm_line[dipole].sin_z == xm_line[dipole].sin_z
                assert pm_line[dipole].angle == xm_line[dipole].angle
            else:
                assert pm_line[dipole].length == xm_line[dipole].length
                assert pm_line[dipole].hxl == xm_line[dipole].hxl
                assert pm_line[dipole].hyl == xm_line[dipole].hyl
                assert pm_line[dipole].knl[0] == xm_line[dipole].knl[0]
                assert pm_line[dipole].ksl[0] == xm_line[dipole].ksl[0]
        except:
            print("PM", dipole, pm_line[dipole].to_dict())
            print("XM", dipole, xm_line[dipole].to_dict())


#### Check if some quadrupoles differ 

In [10]:
quadrupole_names = [x for x in pm_line.element_names if 'mq' in x]
for i, quadrupole in enumerate(quadrupole_names):
    class_element = pm_line[quadrupole].to_dict()['__class__']
    if class_element != 'Marker' and class_element != 'Drift':
        try:
            assert pm_line[quadrupole].length == xm_line[quadrupole].length
            assert pm_line[quadrupole].hxl == xm_line[quadrupole].hxl
            assert pm_line[quadrupole].hyl == xm_line[quadrupole].hyl
            assert np.allclose(pm_line[quadrupole].knl[1], xm_line[quadrupole].knl[1])
            assert np.allclose(pm_line[quadrupole].ksl[1], xm_line[quadrupole].ksl[1])
        except:
            print("PM", quadrupole, pm_line[quadrupole].to_dict())
            print("XM", quadrupole, xm_line[quadrupole].to_dict())

PM mqtlh.a6r3.b1..1 {'__class__': 'Multipole', 'order': 1, 'inv_factorial_order': 1.0, 'length': 0.65, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([ 0.        , -0.00159872]), 'ksl': array([0., 0.])}
XM mqtlh.a6r3.b1..1 {'__class__': 'Multipole', 'order': 1, 'inv_factorial_order': 1.0, 'length': 0.65, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([ 0.       , -0.0015947]), 'ksl': array([0., 0.])}
PM mqtlh.a6r3.b1..2 {'__class__': 'Multipole', 'order': 1, 'inv_factorial_order': 1.0, 'length': 0.65, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([ 0.        , -0.00159872]), 'ksl': array([0., 0.])}
XM mqtlh.a6r3.b1..2 {'__class__': 'Multipole', 'order': 1, 'inv_factorial_order': 1.0, 'length': 0.65, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([ 0.       , -0.0015947]), 'ksl': array([0., 0.])}
PM mqtlh.b6r3.b1..1 {'__class__': 'Multipole', 'order': 1, 'inv_factorial_order': 1.0, 'length': 0.65, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 

#### Check if some sextupoles differ

In [11]:
sextupoles_names = [x for x in pm_line.element_names if 'ms' in x]
for i, sextupole in enumerate(sextupoles_names):
    class_element = pm_line[sextupole].to_dict()['__class__']
    if class_element != 'Marker' and class_element != 'Drift':
        try:
            assert pm_line[sextupole].length == xm_line[sextupole].length
            assert pm_line[sextupole].hxl == xm_line[sextupole].hxl
            assert pm_line[sextupole].hyl == xm_line[sextupole].hyl
            assert np.allclose(pm_line[sextupole].knl[2], xm_line[sextupole].knl[2])
            assert np.allclose(pm_line[sextupole].ksl[2], xm_line[sextupole].ksl[2])
        except:
            print("PM", sextupole, pm_line[sextupole].to_dict())
            print("XM", sextupole, xm_line[sextupole].to_dict())

PM ms.11r3.b1 {'__class__': 'Multipole', 'order': 2, 'inv_factorial_order': 0.5, 'length': 0.369, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([0.        , 0.        , 0.02859794]), 'ksl': array([0., 0., 0.])}
XM ms.11r3.b1 {'__class__': 'Multipole', 'order': 2, 'inv_factorial_order': 0.5, 'length': 0.369, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([0.        , 0.        , 0.02810375]), 'ksl': array([0., 0., 0.])}
PM ms.12r3.b1 {'__class__': 'Multipole', 'order': 2, 'inv_factorial_order': 0.5, 'length': 0.369, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([ 0.        ,  0.        , -0.05953503]), 'ksl': array([0., 0., 0.])}
XM ms.12r3.b1 {'__class__': 'Multipole', 'order': 2, 'inv_factorial_order': 0.5, 'length': 0.369, 'hxl': 0.0, 'hyl': 0.0, 'radiation_flag': 0, 'knl': array([ 0.        ,  0.        , -0.05414635]), 'ksl': array([0., 0., 0.])}
PM ms.13r3.b1 {'__class__': 'Multipole', 'order': 2, 'inv_factorial_order': 0.5, 'length': 0.369, 'hxl

#### Check if some octupoles differ

In [12]:
octupoles_names = [x for x in pm_line.element_names if "mo" in x]
for i, octupole in enumerate(octupoles_names):
    class_element = pm_line[octupole].to_dict()["__class__"]
    if class_element != "Marker" and class_element != "Drift":
        try:
            assert pm_line[octupole].length == xm_line[octupole].length
            assert pm_line[octupole].hxl == xm_line[octupole].hxl
            assert pm_line[octupole].hyl == xm_line[octupole].hyl
            assert np.allclose(pm_line[octupole].knl[3], xm_line[octupole].knl[3])
            assert np.allclose(pm_line[octupole].ksl[3], xm_line[octupole].ksl[3])
        except:
            print("PM", octupole, pm_line[octupole].to_dict())
            print("XM", octupole, xm_line[octupole].to_dict())

### Plot the two lines

In [17]:
tw_xm_pd = xm_line.twiss()  # .to_pandas()
tw_pm_pd = pm_line.twiss().reverse()  # .to_pandas()

plot_all(tw_pm_pd, tw_xm_pd, "Pymask machine (b1)", "Xmask machine (b1)")